# <font color="#5fa8d3"> Yolo model


## <font color="#62b6cb"> Table of Content <a name="ToC"></a>
0. [Libraries Importation, Parameters & Funtions Definition](#id0)<br>
1. [Preparation of the data](#id1)<br>
    1.1 [Copy Images to Yolo Folder](#id11)<br>
    1.2 [Obtain the labels for YOLO](#id12)<br>


## <font color="#62b6cb"> 0. Libraries Importation, Parameters & Funtions Definition <a name="id0"></a>

In [ ]:
# cosas que poner en el config

'./Notebooks/data.yaml'

In [11]:
import json
import os
from ultralytics import YOLO
import shutil
import yaml # for importing a yaml file

In [2]:
current_directory = os.getcwd()
last_folder = os.path.basename(current_directory)
    
if last_folder != "project-danielteresa":
    while last_folder != "project-danielteresa":
        parent_directory = os.path.dirname(current_directory)
        last_folder = os.path.basename(parent_directory)

        os.chdir(parent_directory)
        print(f"Changed directory to: {parent_directory}")
else:
    print("Already in the project root directory.")

# our modules
from src.mymodule import * # for importing our functions

Changed directory to: c:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa


**Configuration Variables**

In [4]:
# Load the YAML file
with open('./config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Function to set variables globally and store their names
def set_variables(config, prefix='', var_dict={}):
    for key, value in config.items():
        if isinstance(value, dict):
            set_variables(value, prefix + key + '_', var_dict)
        else:
            globals()[prefix + key] = value
            var_dict[prefix + key] = value
    return var_dict

# Set variables globally and get a dictionary of the set variables
set_vars = set_variables(config)

# Print all the variables that were set
print("Variables set from YAML file:")
for var_name, var_value in set_vars.items():
    print(f"{var_name}: {var_value}")

Variables set from YAML file:
seed: 123
color1: #62b6cb
color2: #fb8500
color3: #023047
color4: #FFB703
path_annotations: ./Data
path_train: ./Data/train/original
path_train_train: ./Data/train/train
path_train_aug: ./Data/train/train_aug
path_train_val: ./Data/train/val
path_test: ./Data/test
path_yolo: ./Data/Yoloimages/
kaggle_train_annotations: annotations_train.json
train_annotations_name_temp: annotations_train_temp.json
train_annotations_name: annotations_train_updated.json
aug_train_annotations_name: annotations_train_updated_aug.json
val_annotations_name: annotations_val_updated.json
kaggle_test_annotations: annotations_test.json
test_annotations_name_temp: annotations_test_temp.json
test_annotations_name: annotations_test_updated.json
weights_yolo_path: ./yolo_weights


## <font color="#62b6cb"> 1. Preparation of data <a name="id1"></a>

### <font color="#62b6cb"> 1.1 Copy Images to Yolo Folder  <a name="id11"></a> 

In [4]:
# Create the folders if they don't exist
directories = [
    os.path.join(path_yolo, "train/images"),
    os.path.join(path_yolo, "train/labels"),
    os.path.join(path_yolo, "val/images"),
    os.path.join(path_yolo, "val/labels"),
    os.path.join(path_yolo, "test/images"),
    os.path.join(path_yolo, "test/labels")
]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Remove everything that exists in the folders
for directory in [    os.path.join(path_yolo, "train/images"),
                      os.path.join(path_yolo, "train/labels"),
                      os.path.join(path_yolo, "val/images"),
                      os.path.join(path_yolo, "val/labels"),
                      os.path.join(path_yolo, "test/images"),
                      os.path.join(path_yolo, "test/labels")]:
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove the directory and its contents
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

# Copy images from train augmented to Yolo train folder
shutil.copytree(path_train_aug, os.path.join(path_yolo, "train/images"), dirs_exist_ok=True)
shutil.copytree(path_train_val, os.path.join(path_yolo, "val/images"), dirs_exist_ok=True)
shutil.copytree(path_test, os.path.join(path_yolo, "test/images"), dirs_exist_ok=True)

print("Folders have been cleaned and files have been copied successfully.")

Folders have been cleaned and files have been copied successfully.


Check the annotations and id of the coco jsons

In [5]:
print("Check that annotation for training is correct")
print(validate_coco_dataset(os.path.join(path_annotations,aug_train_annotations_name), os.path.join(path_yolo,"train/images")))

print("Check that annotation for validation is correct")
print(validate_coco_dataset(os.path.join(path_annotations,val_annotations_name), os.path.join(path_yolo,"val/images")))

print("Check that annotation for test is correct")
print(validate_coco_dataset(os.path.join(path_annotations,test_annotations_name), os.path.join(path_yolo,"test/images")))

Check that annotation for training is correct
True
Check that annotation for validation is correct
True
Check that annotation for test is correct
True


Some of the points of the polygons are out of the range of the width and the height

In [29]:
# Some of the points of the polygons are out of the range of the images

# train
process_coco_annotations(os.path.join(path_annotations,aug_train_annotations_name),
                          os.path.join(path_annotations,aug_train_annotations_name))

# val
process_coco_annotations(os.path.join(os.path.join(path_annotations,val_annotations_name)), 
                         os.path.join(os.path.join(path_annotations,val_annotations_name)) )

### <font color="#62b6cb"> 1.2 Obtain the labels for YOLO  <a name="id12"></a> 

Obtain the yolo txt for each images from the coco data annotations using the conver_coco from ultralytics. The annotations json to convert shoul be in a folder with that json in it

In [5]:
# Obtain the yolo labels
# train
convert_coco_to_yolo_segmentation(path_annotations, aug_train_annotations_name, path_yolo, 'train')
# val
convert_coco_to_yolo_segmentation(path_annotations, val_annotations_name, path_yolo, 'val')
# test
convert_coco_to_yolo_segmentation(path_annotations, test_annotations_name, path_yolo, 'test')

Annotations C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\Data\Yoloimages\train\annotations_train_updated_aug.json: 100%|██████████| 46485/46485 [01:25<00:00, 546.13it/s]

COCO data converted successfully.
Results saved to C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\Data\Yoloimages\aux


Removed auxiliary directory ./Data/Yoloimages/aux
Yolo labels saved in ./Data/Yoloimages/train\labels



Annotations C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\Data\Yoloimages\val\annotations_val_updated.json: 100%|██████████| 2324/2324 [00:03<00:00, 738.56it/s]

COCO data converted successfully.
Results saved to C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\Data\Yoloimages\aux


Removed auxiliary directory ./Data/Yoloimages/aux
Yolo labels saved in ./Data/Yoloimages/val\labels



Annotations C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\Data\Yoloimages\test\annotations_test_updated.json: 100%|██████████| 2324/2324 [00:03<00:00, 753.90it/s]

COCO data converted successfully.
Results saved to C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\Data\Yoloimages\aux


Removed auxiliary directory ./Data/Yoloimages/aux
Yolo labels saved in ./Data/Yoloimages/test\labels



# TODAVIA NO CAMBIADO

In [12]:
def select_images(source_folder, destination_folder, num_images, images_folder, labels_folder):
    # Eliminar la carpeta de destino si ya existe
    if os.path.exists(destination_folder):
        shutil.rmtree(destination_folder)
    
    # Crear las rutas completas para las carpetas de imágenes y etiquetas
    images_folder_path = os.path.join(source_folder, images_folder)
    labels_folder_path = os.path.join(source_folder, labels_folder)

    destination_images_folder = os.path.join(destination_folder, images_folder)
    destination_labels_folder = os.path.join(destination_folder, labels_folder)

    # Crear las carpetas de destino
    os.makedirs(destination_images_folder)
    os.makedirs(destination_labels_folder)

    # Obtener la lista de imágenes y seleccionar un subconjunto aleatorio
    images = os.listdir(images_folder_path)
    selected_images = random.sample(images, num_images)

    # Copiar las imágenes y sus etiquetas correspondientes
    for image in selected_images:
        image_name = os.path.splitext(image)[0]
        source_image_path = os.path.join(images_folder_path, image)
        destination_image_path = os.path.join(destination_images_folder, image)
        
        source_label_path = os.path.join(labels_folder_path, f"{image_name}.txt")
        destination_label_path = os.path.join(destination_labels_folder, f"{image_name}.txt")
        
        shutil.copy(source_image_path, destination_image_path)
        if os.path.exists(source_label_path):
            shutil.copy(source_label_path, destination_label_path)

# Configuración de carpetas
train_folder = './Data/Yoloimages/train'
val_folder = './Data/Yoloimages/val'

destination_train_folder = './Data/Yoloimages/train_prueba'
destination_val_folder = './Data/Yoloimages/val_prueba'

# Seleccionar 20 imágenes de cada carpeta
select_images(train_folder, destination_train_folder, 20, 'images', 'labels')
select_images(val_folder, destination_val_folder, 20, 'images', 'labels')




The yaml file is created for the yolo code.

In [20]:
with open(os.path.join(path_annotations,val_annotations_name), 'r') as f:
    coco_data = json.load(f)

names = [class_name['name'] for class_name in coco_data["categories"]]



# Specify the paths and information
actual_path = os.getcwd()
train_path = os.path.join(actual_path, 'CustomImages/train_prueba/images')
val_path = os.path.join(actual_path, 'CustomImages/val_prueba/images')

names_categories = [class_name['name'] for class_name in coco_data["categories"]]
nc = len(names)
file_path = './Notebooks/data.yaml'

# Create the YAML file
create_yaml_file(file_path, train_path, val_path, nc, names)


./Notebooks/data.yaml


In [36]:
# %reset  # eliminate variables defined in the session

Don't know how to reset  #, please run `%reset?` for details
Don't know how to reset  eliminate, please run `%reset?` for details
Don't know how to reset  variables, please run `%reset?` for details
Don't know how to reset  defined, please run `%reset?` for details
Flushing input history
Don't know how to reset  the, please run `%reset?` for details
Don't know how to reset  session, please run `%reset?` for details


In [7]:
if not os.path.exists(weights_yolo_path):
    os.makedirs(weights_yolo_path)
model = YOLO(os.path.join(weights_yolo_path, "yolov8m-seg.pt"))

In [ ]:
epochs	100	
patience	100
batch	16	Batch size, with three modes: set as an integer (e.g., batch=16), auto mode for 60% GPU memory utilization (batch=-1), or auto mode with specified utilization fraction (batch=0.70).
imgsz	640	
lr0	0.01	Initial learning rate (i.e. SGD=1E-2, Adam=1E-3) . Adjusting this value is crucial for the optimization process, influencing how rapidly model weights are updated.
weight_decay	
dropout

USAR ADAM AUIDA A MAS RAPIDA CONVERGENCIA

verbose =False

In [ ]:
space = {
    'lr0': [0.001, 0.01],  # Initial learning rate
}

# Tune the model
model.tune(space=space, data='./Notebooks/data.yaml')

In [17]:
model.train(data="data.yaml", epochs=3, optimizer='Adam', task='segment')

Tuner: Initialized Tuner instance with 'tune_dir=C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\runs\segment\tune9'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01}


In [9]:

! pip install -U ultralytics

In [8]:
! python.exe -m pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 660.6 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.8 MB 2.0 MB/s eta 0:00:01
   -------- ------------------------------- 0.4/1.8 MB 3.1 MB/s eta 0:00:01
   ------------ --------------------------- 0.6/1.8 MB 3.0 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/1.8 MB 3.2 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/1.8 MB 3.4 MB/s eta 0:00:01
   ------------------------ --------------- 1.1/1.8 MB 3.3 MB/s eta 0:00:01
   ------------------------------ --------- 1.4/1.8 MB 3.6 MB/s eta 0:00:01
   ----------------------------------- ---- 1.6/1.8 MB 3.8 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 3.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24

In [19]:
results = model.predict("03bda226ad62553c0c73.jpg")


image 1/1 c:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\03bda226ad62553c0c73.jpg: 480x640 300 met_scratchs, 501.6ms
Speed: 5.4ms preprocess, 501.6ms inference, 145.0ms postprocess per image at shape (1, 3, 480, 640)


In [34]:
results[0].masks

ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,

In [22]:
# Realizar predicciones
results = model.predict("03bda226ad62553c0c73.jpg", conf=0.9, iou=0.7)

# Visualizar resultados
for result in results:
    img = result.plot(show=True, labels=True, masks=True, boxes=True)
    plt.imshow(img)
    plt.show()


image 1/1 c:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\03bda226ad62553c0c73.jpg: 480x640 132 met_scratchs, 412.3ms
Speed: 0.0ms preprocess, 412.3ms inference, 58.0ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 640x480 with 1 Axes>